In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from experiments.util import load_data

from sklearn.model_selection import train_test_split

In [4]:
X_data, y_data = load_data()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, random_state=42)

#### PCA

In [6]:
from sklearn.decomposition import PCA

In [7]:
%%time
pca = PCA(n_components=0.9).fit(X_train.reshape(-1, 32 * 32 * 3))

print(X_train.reshape(-1, 32 * 32 * 3).shape)
print(pca.transform(X_train.reshape(-1, 32 * 32 * 3)).shape)

(50000, 3072)
(50000, 99)
CPU times: user 6min 55s, sys: 24.2 s, total: 7min 19s
Wall time: 1min 30s


In [8]:
%%time

from sklearn.pipeline import Pipeline

pca_lr = Pipeline([
    ('pca', PCA(n_components=0.9)),
    ('clf', LogisticRegression())
])

pca_lr.fit(X_train.reshape(-1, 32 * 32 * 3), y_train)

y_val_hat = pca_lr.predict(X_val.reshape(-1, 32 * 32 * 3))
accuracy_score(y_val_hat, y_val)

/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CPU times: user 7min 20s, sys: 22.1 s, total: 7min 42s
Wall time: 1min 30s


0.4009

In [9]:
%%time

from sklearn.svm import SVC

pca_svm = Pipeline([
    ('pca', PCA(n_components=0.9)),
    ('clf', SVC(kernel='rbf'))
])

pca_svm.fit(X_train.reshape(-1, 32 * 32 * 3), y_train)

y_train_hat = pca_svm.predict(X_train.reshape(-1, 32 * 32))
print("Train: ", accuracy_score(y_train_hat, y_train))

y_val_hat = pca_svm.predict(X_val.reshape(-1, 32 * 32 * 3))
print("Val:   ", accuracy_score(y_val_hat, y_val))

/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


CPU times: user 15min 26s, sys: 31.6 s, total: 15min 58s
Wall time: 10min 8s


0.5411

#### Random Forest PCA

In [10]:
%%time

from sklearn.ensemble import RandomForestClassifier

pca_rf = Pipeline([
    ('pca', PCA(n_components=0.9)),
    ('clf', RandomForestClassifier())
])

pca_rf.fit(X_train.reshape(-1, 32 * 32 * 3), y_train)

y_train_hat = pca_rf.predict(X_train.reshape(-1, 32 * 32))
print("Train: ", accuracy_score(y_train_hat, y_train))

y_val_hat = pca_rf.predict(X_val.reshape(-1, 32 * 32 * 3))
print("Val:   ", accuracy_score(y_val_hat, y_val))

/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-packages/sklearn/pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


CPU times: user 6min 30s, sys: 17.6 s, total: 6min 48s
Wall time: 2min 14s


0.4645

#### Hyperparameter

In [14]:
grid = {
    'pca__n_components': [0.8, 0.85, 0.9, 0.95, 0.975]
}

In [15]:
from sklearn.model_selection import GridSearchCV

pca_lr_gs = GridSearchCV(pca_lr, param_grid=grid)

pca_lr_gs.fit(X_train.reshape(-1, 32 * 32 * 3), y_train)

y_train_hat = pca_lr_gs.predict(X_train.reshape(-1, 32 * 32))
print("Train: ", accuracy_score(y_train_hat, y_train))

y_val_hat = pca_lr_gs.predict(X_val.reshape(-1, 32 * 32 * 3))
print("Val:   ", accuracy_score(y_val_hat, y_val))


/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-p

ValueError: X has 1024 features, but PCA is expecting 3072 features as input.